<a href="https://colab.research.google.com/github/aasharib/CV_Assignment_3/blob/main/CV_Assignment_3_simple_vgg_val_84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#!unzip -q /content/drive/MyDrive/Semester1/CV/Assignment/3/intel-image-classification.zip -d /content/assgn3

In [3]:
from sklearn.datasets import load_files
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [4]:
trainData = load_files('/content/assgn3/seg_train/seg_train',load_content=False)
testData = load_files('/content/assgn3/seg_test/seg_test',load_content=False)
#validationData = load_files('/content/assgn3/seg_pred/seg_pred',load_content=False)

In [5]:
trainData

{'DESCR': None,
 'filenames': array(['/content/assgn3/seg_train/seg_train/glacier/6416.jpg',
        '/content/assgn3/seg_train/seg_train/buildings/5658.jpg',
        '/content/assgn3/seg_train/seg_train/street/7656.jpg', ...,
        '/content/assgn3/seg_train/seg_train/sea/13907.jpg',
        '/content/assgn3/seg_train/seg_train/sea/3467.jpg',
        '/content/assgn3/seg_train/seg_train/forest/14133.jpg'],
       dtype='<U55'),
 'target': array([2, 0, 5, ..., 4, 4, 1]),
 'target_names': ['buildings',
  'forest',
  'glacier',
  'mountain',
  'sea',
  'street']}

In [6]:
trainDataLabel = trainData.target

In [7]:
def readAndPreprocessImage(imgFile):
    curImg = np.asarray(Image.open(imgFile).resize((150,150)))
    if curImg is not None:
        #print("Done", imgFile)
        return curImg 

In [8]:
trainDataImgArr = np.array(list(map(readAndPreprocessImage, trainData.filenames)))

In [9]:
(trainingDataImgX, validationDataImgX, trainingDataLabelY, validationDataLabelY) = train_test_split(trainDataImgArr, trainDataLabel, test_size=0.2, random_state=42)

2
3


In [10]:

fig, ax = plt.subplots(1,2,figsize=(6,6))
ax[0].imshow(trainingDataImgX[0])
ax[1].imshow(trainingDataImgX[1])
print(trainingDataLabelY[0])
print(trainingDataLabelY[1])

In [11]:
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout
from keras.models import Sequential

vgg16 = Sequential()
vgg16.add(Conv2D(64, kernel_size=3, padding='same', activation='relu',
                 input_shape=(150, 150, 3)))
vgg16.add(Conv2D(64, kernel_size=3, padding='same', activation='relu'))
vgg16.add(MaxPooling2D(pool_size=2, strides=2))
vgg16.add(Dropout(0.25))

vgg16.add(Conv2D(128, kernel_size=3, padding='same', activation='relu'))
vgg16.add(Conv2D(128, kernel_size=3, padding='same', activation='relu'))
vgg16.add(MaxPooling2D(pool_size=2, strides=2))
vgg16.add(Dropout(0.25))

vgg16.add(Conv2D(256, kernel_size=3, padding='same', activation='relu'))
vgg16.add(Conv2D(256, kernel_size=3, padding='same', activation='relu'))
vgg16.add(Conv2D(256, kernel_size=3, padding='same', activation='relu'))
vgg16.add(MaxPooling2D(pool_size=2, strides=2))
vgg16.add(Dropout(0.25))

vgg16.add(Conv2D(512, kernel_size=3, padding='same', activation='relu'))
vgg16.add(Conv2D(512, kernel_size=3, padding='same', activation='relu'))
vgg16.add(Conv2D(512, kernel_size=3, padding='same', activation='relu'))
vgg16.add(MaxPooling2D(pool_size=2, strides=2))
vgg16.add(Dropout(0.25))

vgg16.add(Conv2D(512, kernel_size=3, activation='relu'))
vgg16.add(Conv2D(512, kernel_size=3, activation='relu'))
vgg16.add(Conv2D(512, kernel_size=3, activation='relu'))
vgg16.add(MaxPooling2D(pool_size=2, strides=2))
vgg16.add(Dropout(0.25))

vgg16.add(Flatten())

vgg16.add(Dense(4096, activation='relu'))
vgg16.add(Dense(4096, activation='relu'))
vgg16.add(Dense(4096, activation='relu'))

# the data we are using has 6 classes
vgg16.add(Dense(6, activation='softmax'))

In [12]:
#vgg16.summary()

In [13]:
from keras.optimizers import SGD
sgd = SGD(lr=0.001, decay=1e-7, momentum=.9)
vgg16.compile(loss='categorical_crossentropy', 
              optimizer=sgd,
              metrics=['accuracy'])

In [14]:
trainingDataLabelY[0]

2

In [15]:
from keras.utils import to_categorical
trainingDataLabelOHE = to_categorical(trainingDataLabelY)
validationDataLabelOHE = to_categorical(validationDataLabelY)

In [16]:
trainingDataLabelOHE[0]

array([0., 0., 1., 0., 0., 0.], dtype=float32)

In [17]:
H = vgg16.fit(trainingDataImgX, trainingDataLabelOHE, batch_size=128, epochs=50, validation_data=(validationDataImgX, validationDataLabelOHE))

Epoch 1/50
88/88 [==============================] - 91s 938ms/step - loss: 1.7838 - accuracy: 0.1928 - val_loss: 1.5012 - val_accuracy: 0.3684
Epoch 2/50
88/88 [==============================] - 74s 846ms/step - loss: 1.3739 - accuracy: 0.4363 - val_loss: 1.2578 - val_accuracy: 0.4735
Epoch 3/50
88/88 [==============================] - 74s 846ms/step - loss: 1.1371 - accuracy: 0.5434 - val_loss: 1.0884 - val_accuracy: 0.5807
Epoch 4/50
88/88 [==============================] - 74s 846ms/step - loss: 1.0565 - accuracy: 0.5739 - val_loss: 1.0579 - val_accuracy: 0.5650
Epoch 5/50
88/88 [==============================] - 74s 845ms/step - loss: 0.9975 - accuracy: 0.6005 - val_loss: 1.0376 - val_accuracy: 0.6046
Epoch 6/50
88/88 [==============================] - 74s 845ms/step - loss: 0.9722 - accuracy: 0.6069 - val_loss: 1.0953 - val_accuracy: 0.5586
Epoch 7/50
88/88 [==============================] - 74s 845ms/step - loss: 0.9262 - accuracy: 0.6309 - val_loss: 0.9971 - val_accuracy: 0.6274

In [18]:
# save the model's trained weights
vgg16.save_weights('vgg16_trained_50epochs_normal_vgg_with_validation.h5')

In [19]:
!cp /content/vgg16_trained_normal_vgg.h5 /content/drive/MyDrive/Semester1/CV/Assignment/3

In [20]:
import matplotlib.pyplot as plt
N = np.arange(0, 50)
plt.style.use('ggplot')
plt.figure()
plt.plot(N, H.history['loss'], label='train_loss')
plt.plot(N, H.history['val_loss'], label='val_loss')
plt.plot(N, H.history['accuracy'], label='train_accuracy')
plt.plot(N, H.history['val_accuracy'], label='val_accuracy')
plt.title('Training loss and accuracy')
plt.xlabel('Epoch #')
plt.ylabel('Loss/Accuracy')
plt.legend()
plt.show()

In [21]:
testDataImgArr = np.array(list(map(readAndPreprocessImage, testData.filenames)))
testDataLabel = testData.target
testDataLabelOHE = to_categorical(testDataLabel)

In [22]:
'''CONFUSION MATRIX'''
# Making prediction
predicatedTestDataY = vgg16.predict_classes(testDataImgArr)
actualTestDataY = np.argmax(testDataLabelOHE, axis=-1)

# Plotting the confusion matrix
from sklearn.metrics import confusion_matrix
confusionMatrix = confusion_matrix(actualTestDataY, predicatedTestDataY)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [23]:
print(confusionMatrix)

[[363   9   0   6   7  52]
 [  0 460   3   6   1   4]
 [  2   5 427  77  36   6]
 [  0   5  65 420  35   0]
 [  7   4  22  14 453  10]
 [ 48  20   0   4   3 426]]
